In [19]:
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

import numpy as np
import pandas as pd
import category_encoders as ce
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import plotly.express as px

# Load Data

In [8]:
player = "../data/Players.csv"
player_data = "../data/player_data.csv"
season_stat = "../data/Seasons_Stats.csv"

df_player = pd.read_csv(player)
df_season = pd.read_csv(season_stat)
print(df_season.shape, df_player.shape)

(24691, 53) (3922, 8)


In [10]:
#MAKING DICT TO MAP OVER NEW COLUMN
dict_games = {
    1950:68,
    1951:66,
    1952:70,
    1953:72,
    1954:72,
    1955:72,
    1956:72,
    1957:72,
    1958:72,
    1959:75,
    1960:79,
    1961:80,
    1962:80,
    1963:80,
    1964:80,
    1965:80,
    1966:81,
    1967:82,
    1968:82,
    1969:82,
    1970:82,
    1971:82,
    1972:82,
    1973:82,
    1974:82,
    1975:82,
    1976:82,
    1977:82,
    1978:82,
    1979:82,
    1980:82,
    1981:82,
    1982:82,
    1983:82,
    1984:82,
    1985:82,
    1986:82,
    1987:82,
    1988:82,
    1989:82,
    1990:82,
    1991:82,
    1992:82,
    1993:82,
    1994:82,
    1995:82,
    1996:82,
    1997:82,
    1998:82,
    1999:50, #NBA Lockout
    2000:82,
    2001:82,
    2002:82,
    2003:82,
    2004:82,
    2005:82,
    2006:82,
    2007:82,
    2008:82,
    2009:82,
    2010:82,
    2011:66, #NBA Lockout
    2012:82,
    2013:82,
    2014:82,
    2015:82,
    2016:82,
    2017:82,
    2018:82,
}

In [15]:
# Using dict_games to map over a new series and normalize my y('G')
df_season['season_length'] = df_season['Year'].map(dict_games)
df_season['season_pct_played'] = df_season['G']/df_season['season_length']

# Making train and test df
testing = df_season[df_season['Year']>= 2005]
training = df_season[df_season['Year'] < 2005]
testing.shape, training.shape

# Define X and y
target = 'season_pct_played'
features = ['Pos', 'BLK', 'STL', 'AST','Age', 'TRB', 'PTS']

#DF X and y
train, val = train_test_split(training, random_state=42) #Splitting merged df

X_train = train[features]
y_train = train[target]

X_val = val[features]
y_val = val[target]

X_test = testing[features]
y_test = testing[target]

X_test_cv = training[features]
y_train_cv = testing[target]

print(f'X_train: {X_train.shape}')
print(f'X_val: {X_val.shape}')
print(f'X_test: {X_test.shape}')

X_train: (12765, 7)
X_val: (4256, 7)
X_test: (7603, 7)


In [30]:
gb = make_pipeline(
    ce.OrdinalEncoder(),
    StandardScaler(),
    SimpleImputer(strategy='median'), 
    XGBRegressor(
        n_estimators=100, 
        n_jobs=-1,
        max_depth=15,
        learning_rate=0.1,
        gamma=0.2
    )
)

gb.fit(X_train, y_train)
y_pred = gb.predict(X_val)
print('Gradient Boosting R^2', r2_score(y_val, y_pred))

/Users/mph/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/mph/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning:

Series.base is deprecated and will be removed in a future version



[19:32:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Gradient Boosting R^2 0.8433435176566212


In [31]:
#TEST SCORES
y_pred = gb.predict(X_test)
print('Gradient Boosting R^2', r2_score(y_test, y_pred))
print('Gradient Boosting RMSE', np.sqrt(mean_squared_error(y_test, y_pred)))

Gradient Boosting R^2 0.8118982963318925
Gradient Boosting RMSE 0.1414205364870305
